In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mytool import uav_traffic_tool
tool = uav_traffic_tool.Wiz()

In [ ]:
dataset_information={
    'day':'24Oct',
    'month':'10',
    'drone':'2',
    'time_period':'0900_0930',
    'road':'Panepistimiou',
    'intersection':'Panepistimiou-Omirou'
    }

nickname = f'Drone{dataset_information.get('drone')}_{dataset_information.get('day')}_{dataset_information.get('time_period')}'
file_path= fr'C:\Users\kpourg01\Desktop\Work\Code\pNEUMA\datasets\{dataset_information.get('road')}/2018{dataset_information.get('month')}{dataset_information.get('day')[:2]}_d{dataset_information.get('drone')}_{dataset_information.get('time_period')}.csv'

In [ ]:
#sampling period must be integer multiple of 0.04
period_dataloading=0.2
time_decimals=1
#===========================================================================================
from tests.dataload_example import get_info_pneuma as infoloader 
raw_vd = infoloader(file_path=file_path, sampling_period=period_dataloading, time_decimals=time_decimals, position_smoothing=2)

In [ ]:
period_flowcount=1
#----------------------------------------------------
time_axis = np.round(np.arange(min(min(set) for set in raw_vd.get('time')),max(max(set) for set in raw_vd.get('time'))+period_flowcount,period_flowcount),decimals=time_decimals).tolist()
#----------------------------------------------------
wgs=True
#----------------------------------------------------
ll_lat,ll_lon = 37.97811671602297, 23.733975874806358
lr_lat,lr_lon = 37.97876143771719, 23.735210640421347
ur_lat,ur_lon = 37.97938790250702, 23.734674127747873
ul_lat,ul_lon = 37.97874587304865, 23.73345028699124
#----------------------------------------------------
clat,clon = 37.97866950849114, 23.734362398006162
#----------------------------------------------------
bbox = [(ll_lat,ll_lon), 
        (lr_lat,lr_lon),
        (ur_lat,ur_lon),
        (ul_lat,ul_lon)]
#----------------------------------------------------
spatio_temporal_info =  {'wgs':wgs,'bbox':bbox,'x center': clon,'y center': clat,'time axis': time_axis}

In [ ]:
dataloader = tool.dataloader(raw_vd, wgs, bbox)
filtered_vd = dataloader.get_filtered_vd(cursed_ids=[208,1442,75,2450,140,1151,1469,628,2941,154,917])
analysis = tool.analysis(filtered_vd,spatio_temporal_info)
visualization = tool.visualization(filtered_vd,spatio_temporal_info)

In [ ]:
od_pairs = analysis.get_od_pairs()
valid_od_pairs = [(1,3),(1,2),(4,3),(4,2)]
visualization.draw_trajectories_od(valid_od_pairs)

In [ ]:
vd_13_12 = analysis.get_od_vd(desirable_pairs=[(1,3),(1,2)])
analysis_13_12 = tool.analysis(vd_13_12,spatio_temporal_info)
visualization_13_12 = tool.visualization(vd_13_12,spatio_temporal_info)
#----------------------------------------------------------------------
vd_43_42 = analysis.get_od_vd(desirable_pairs=[(4,3),(4,2)])
analysis_43_42 = tool.analysis(vd_43_42,spatio_temporal_info)
visualization_43_42 = tool.visualization(vd_43_42,spatio_temporal_info)

In [ ]:
lane_info_13_12 = analysis_13_12.get_lane_info(flow_direction='up')

In [ ]:
lane_info_43_42 = analysis_43_42.get_lane_info(flow_direction='left')

In [ ]:
sorted_id_13_12 =analysis_13_12.get_sorted_id()
gaps_13_12 = analysis_13_12.get_gaps()
#-=============================================
sorted_id_43_42 = analysis_43_42.get_sorted_id()
gaps_43_42 = analysis_43_42.get_gaps()

In [ ]:
detector_positions_13_12 = [
                            (37.978598063851116, 23.73433701749522),
                            (37.97861444910225, 23.73438060338954),
                            (37.978632948574926, 23.734412119343897),
                            (37.978652505155274, 23.734447658611572),
                            (37.978666776170066, 23.734481856774803)
                            ]
#---------------------------
flow_info_13_12 = analysis_13_12.get_flow_info(detector_positions_13_12)
flow_13_12,normalized_flow_13_12 = analysis_13_12.get_normalized_flow(threshold=10)
#============================
detector_positions_43_42 = [(37.97872544586954, 23.73444095309425)]
#----------------------------
flow_info_43_42 = analysis_43_42.get_flow_info(detector_positions_43_42)
flow_43_42,normalized_flow_43_42 = analysis_43_42.get_normalized_flow(threshold=15)

In [ ]:
print(f'flow counts main: {sum(flow_13_12)}, max vehicles: {len(vd_13_12.get('id'))}')
print(f'\nflow counts secondary: {sum(flow_43_42)}, max vehicles: {len(vd_43_42.get('id'))}')

In [ ]:
plt.figure(figsize=(22,3))
plt.title('Flow vs Time')

plt.plot(time_axis,normalized_flow_13_12,color='purple',alpha=1, label=f'{dataset_information.get('road')} Detector',linewidth=2)
#plt.plot(time_axis,flow_13_12,color='purple',label='raw flow',alpha=0.4)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
plt.plot(time_axis,normalized_flow_43_42,color='orange',alpha=1,label=f'{dataset_information.get('intersection')[dataset_information.get('intersection').index('-')+1:]} Detector',linewidth=2)
#plt.plot(time_axis,flow_43_42,color='orange',label='raw flow',alpha=1)

plt.xlabel('Time (s)')
plt.yticks([0,1],['Detector OFF','Detector ON'])
plt.xticks(np.arange(0,time_axis[-1],25))
plt.ylim(-0.1,2)


activate_zoom = False
#===================================
if activate_zoom:
    lowlim,highlim=275,300
    plt.xlim(lowlim,highlim)
    plt.xticks(np.arange(lowlim,highlim+period_flowcount,period_flowcount))
    analysis_43_42.get_cursed_id(lowlim,highlim)
#===================================

plt.legend()
plt.tight_layout()
plt.show(close=True)

In [ ]:
tlp_13_12 = analysis_13_12.get_traffic_light_phases()
tlp_43_42 = analysis_43_42.get_traffic_light_phases()
#==================================================
cycles = analysis.get_traffic_light_cycles(tlp_13_12,tlp_43_42)

In [ ]:
queue_info_13_12 = analysis_13_12.get_queue_info(speed_threshold=30, gap_threshold=10)
queue_info_43_42 = analysis_43_42.get_queue_info(speed_threshold=30, gap_threshold=10)